In [1]:
#%pip install helper_utils


In [2]:
#%pip install -U langchain

In [3]:
#%pip install pypdf

In [4]:
#%pip install sentence-transformers

In [5]:
#%pip install chromadb

In [6]:
import helper_utils as helpers

In [7]:
#from helper_utils import word_wrap

In [8]:
from pypdf import PdfReader

reader = PdfReader("dataset/2023_Annual_Report.pdf")
pdf_texts = [p.extract_text().strip() for p in reader.pages]

# Filter the empty strings
pdf_texts = [text for text in pdf_texts if text]

print((pdf_texts[0]))

1 Dear shareholders, colleagues, customers, and partners,  
We are living through a time of historic challenge and opportunity. As I write this, the world faces ongoing economic, social , 
and geopolitical volatility. At the same time, we have entered a new age of AI that will fundamentally transform productivity  
for every individual, organization, and industry on earth, and help us address some of our most pressing challenges.  
This next generation of AI will reshape every software category and every business, including our own. Forty -eight years 
after its founding, Microsoft remains a consequential company because time and time again —from PC/Server, to 
Web/Internet, to Cloud/Mob ile—we have adapted to technological paradigm shifts. Today, we are doing so once again, as 
we lead this new era.  
Amid this transformation, our mission to empower every person and every organization on the planet to achieve 
more  remains constant. As a company, we believe we can be the democratizin

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [10]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=0
)
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

#print(word_wrap(character_split_texts[10]))
print(f"\nTotal chunks: {len(character_split_texts)}")


Total chunks: 343


In [11]:

from langchain.text_splitter import SentenceTransformersTokenTextSplitter

In [12]:

token_splitter = SentenceTransformersTokenTextSplitter(chunk_overlap=0, tokens_per_chunk=256)

token_split_texts = []
for text in character_split_texts:
    token_split_texts += token_splitter.split_text(text)

#print(word_wrap(token_split_texts[10]))
print(f"\nTotal chunks: {len(token_split_texts)}")

c:\Users\pmohapatra\GenAI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
modules.json: 100%|██████████| 349/349 [00:00<00:00, 220kB/s]
c:\Users\pmohapatra\GenAI\.venv\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pmohapatra\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate D


Total chunks: 347


In [13]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([token_split_texts[10]]))

modules.json: 100%|██████████| 349/349 [00:00<00:00, 222kB/s]
c:\Users\pmohapatra\GenAI\.venv\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pmohapatra\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
config_sentence_transformer

[[-0.10381157696247101, 0.036902204155921936, -0.03827807679772377, -0.04762120917439461, -0.02947862818837166, 0.013412977568805218, 0.008322598412632942, 0.06288386881351471, 0.029157262295484543, -0.0079110786318779, -0.03119717724621296, 0.04124707728624344, 0.044842351227998734, 0.031169027090072632, 0.037144389003515244, -0.012769141234457493, 0.0025719120167195797, -0.0546116977930069, -0.016872821375727654, 0.022510813549160957, -0.0520002506673336, 0.04188738763332367, -0.026577556505799294, 0.0189029723405838, -0.01753348484635353, 0.04174415022134781, -0.07124286890029907, -0.11644632369279861, 0.00984302256256342, -0.03457752615213394, 0.034627851098775864, -0.018639294430613518, 0.06909224390983582, 0.0816601812839508, 0.006685691419988871, -0.03778970614075661, 0.03195148706436157, 0.06072944030165672, -0.021703965961933136, -0.0837971419095993, -0.006467427127063274, -0.07819971442222595, -0.0680152177810669, 0.032416049391031265, 0.11298935860395432, -0.0169612765312194

In [14]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("microsoft_annual_report", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_split_texts))]

chroma_collection.add(ids=ids, documents=token_split_texts)
chroma_collection.count()

347

In [15]:
query = "What was the total revenue?"

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print((document))
    print('\n')

include legal, including settlements and fines, information technology, human resources, finance, excise taxes, field selling, share d facilities services, customer service and support, and severance incurred as part of a corporate program. each allocation is measured differently based on the specific facts and circumstances of the costs being allocated and is generall y based on relativ e gross margin or relative headcount. segment revenue and operating income were as follows during the periods presented : ( in millions ) year ended june 30, 2023 2022 2021 revenue productivity and business processes $ 69, 274 $ 63, 364 $ 53, 915 intelligent cloud 87, 907 74, 965 59, 728 more personal computing 54, 734 59, 941 54, 445 total $ 211, 915 $ 198, 270 $ 168, 088 operating income productivity and business processes $ 34, 189 $ 29, 690 $ 24, 351


intelligent cloud 37, 884 33, 203 26, 471 more personal computing 16, 450 20, 490 19, 094 total $ 88, 523 $ 83, 383 $ 69, 916 no sales to an individ

In [16]:
import os
import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

openai_client = OpenAI()

In [17]:
def rag(query, retrieved_documents, model="gpt-3.5-turbo"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report."
            "You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [18]:
output = rag(query=query, retrieved_documents=retrieved_documents)

print((output))

The total revenue for the year ended June 30, 2023, as stated in the annual report, was $211,915 million.
